In [2]:
%pip install langchain langchain-community langchain-text-splitters
%pip install chromadb
%pip install sentence-transformers
%pip install bs4 requests
%pip install langchain-huggingface

Note: you may need to restart the kernel to use updated packages.
  Using cached build-1.4.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached pybase64-1.4.3-cp311-cp311-win_amd64.whl.metadata (9.1 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached opentelemetry_api-1.39.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.39.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_sdk-1.39.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached tokenizers-0.22.2-cp39-abi3-win_amd64.whl.metadata (7.4 kB)
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached bcrypt-5.0.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached kubernetes-35.0.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached mmh3-5.2.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached jsonschema-4.26.0-py3-none-any.whl.metadata (7

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.1.53 requires packaging<24.0,>=23.2, but you have packaging 26.0 which is incompatible.


  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached markupsafe-3.0.3-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.4 MB 1.3 MB/s eta 0:00:08
   -- ------------------------------------- 0.5/10.4 MB 1.3 MB/s eta 0:00:08
   -- ------------------------------------- 0.5/10.4 MB 1.3 MB/s eta 0:00:08
   --- ------------------------------------ 0.8/10.4 MB 671.3 kB/s eta 0:00:15
   ---- ----------------------------------- 1.0/10.4 MB 786.4 kB/s eta 0:00:12
   ----- ---------------------------------- 1.3/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.16 requires langchain-core<0.2.0,>=0.1.42, but you have langchain-core 1.2.15 which is incompatible.
langchain 0.1.16 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.7.6 which is incompatible.
langchain-community 0.0.34 requires langchain-core<0.2.0,>=0.1.45, but you have langchain-core 1.2.15 which is incompatible.
langchain-community 0.0.34 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.7.6 which is incompatible.
langchain-text-splitters 0.0.2 requires langchain-core<0.3,>=0.1.28, but you have langchain-core 1.2.15 which is incompatible.
langgraph 0.0.36 requires langchain-core<0.2.0,>=0.1.42, but you have langchain-core 1.2.15 which is incompatible.
transformers 5.2.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


def load_url(url: str):
    loader = WebBaseLoader(
        web_path=url,
        bs_kwargs={
            "parse_only": None
        }
    )
    documents = loader.load()

    if not documents:
        raise Exception(" No content extracted from URL")

    return documents

In [ ]:
def chunk_documents(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = splitter.split_documents(documents)
    return chunks

In [ ]:
def get_embeddings():
    return HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

In [ ]:
def store_in_chroma(chunks, persist_dir="chroma_db"):
    embeddings = get_embeddings()

    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_dir
    )

    vectordb.persist()
    return vectordb

In [ ]:
def ingest_url_to_chroma(url: str):
    print("[+] Loading URL...")
    docs = load_url(url)

    print("[+] Chunking content...")
    chunks = chunk_documents(docs)

    print(f"[+] Total chunks created: {len(chunks)}")

    print("[+] Storing in ChromaDB...")
    vectordb = store_in_chroma(chunks)

    print(" URL successfully indexed into ChromaDB")
    return vectordb

In [ ]:
def get_retriever(persist_dir="chroma_db"):
    embeddings = get_embeddings()

    vectordb = Chroma(
        persist_directory=persist_dir,
        embedding_function=embeddings
    )

    retriever = vectordb.as_retriever(
        search_type="mmr",   # BEST for RAG
        search_kwargs={
            "k": 5,
            "fetch_k": 20
        }
    )
    return retriever

In [ ]:
def query_rag(question: str):
    retriever = get_retriever()
    docs = retriever.get_relevant_documents(question)

    print("\n🔍 Retrieved Context:\n")
    for i, d in enumerate(docs):
        print(f"[Chunk {i+1}] {d.page_content[:300]}...\n")

    return docs

In [ ]:
url = "https://0a39000a039174bb82c6de8a008c0023.web-security-academy.net/"

# Step 1: Ingest
ingest_url_to_chroma(url)

# Step 2: Query
query_rag("Does this page contain phishing indicators or suspicious scripts?")